In [14]:
from scipy.optimize import fsolve
import scipy.optimize as opt
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad,trapz, cumtrapz, odeint, solve_ivp
from scipy import interpolate
import ThermalFrontFormulation as TF
from unpackConfigurations import unpackConfiguration,returnzl,returnll
from matplotlib.collections import LineCollection
import multiprocessing as mp
from collections import defaultdict
from timeit import default_timer as timer
import pandas as pd
from AnalyticCoolingCurves import LfuncN, LfuncAr
from unpackConfigurations import unpackConfiguration
from unpackConfigurationsMK import unpackConfigurationMK
import ThermalFrontFormulation as TF
import os
from LRBv2 import LRBv2

from LengyelReinkeFormulation import *
from LengyelReinkeFormulationMK import returnImpurityFracLengMK
import ThermalFrontFormulation as TF

%load_ext autoreload
%autoreload 2

fontsize = 12

plt.rc('font', size=fontsize) #controls default text size
plt.rc('axes', titlesize=fontsize) #fontsize of the title
plt.rc('axes', labelsize=fontsize) #fontsize of the x and y labels
plt.rc('xtick', labelsize=fontsize) #fontsize of the x tick labels
plt.rc('ytick', labelsize=fontsize) #fontsize of the y tick labels
plt.rc('legend', fontsize=fontsize) #fontsize of the legend

def file_write(data, filename):
# Writes an object to a pickle file.
    
    with open(filename, "wb") as file:
    # Open file in write binary mode, dump result to file
        pkl.dump(data, file)
        
        
        
def file_read(filename):
# Reads a pickle file and returns it.

    with open(filename, "rb") as filename:
    # Open file in read binary mode, dump file to result.
        data = pkl.load(filename)
        
    return data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IO

In [4]:
gridFile = r"C:\Users\Mike\OneDrive\Project\DLS\STEPv10\drsep0mm\balance_v10.nc"
sepadd = 2
res = 500
d = unpackConfigurationMK(File = gridFile, Type = "ou", sepadd = 2, resolution = res, convention = "target_to_midplane")

In [16]:
# results = dict()

#set switches
radios_default = {
    "ionisation": False,  # in development
    "upstreamGrid": False, #if true, source of divertor heat flux comes from radial transport upstream, and Tu is at the midplane. If false, heat flux simply enters at the x point as qi, and Tu is located at the x point. 
    "fluxlim": False,  # if true, turns on a flux limiter with coefficient alpha
}

#set general run parameters
constants_default = {
    "gamma_sheath": 7, #sheath transmittion coefficient for virtual target. Choice does not matter if Tt is low
    "qpllu0": 1e9, # heat flux density at the x point
    "nu" : 1e20, #upstream density
    "nu0" : 1e20,
    "cz0" : 0.05,
    "kappa0" : 2500,
    "mi": 3*10**(-27),
    "echarge": 1.60*10**(-19), 
    "Tt": 0.5, # requested virtual target temp. Aim for low (sub 1eV) Tt
    "Lfunc": LfuncAr, #impurity cooling function
    "alpha": 1000, #flux limiting alpha. Only matters if fluxlim is true
}

d["indexrange"] =  np.linspace(0,len(d["S"][:d["Xpoint"]])*0.98,15) # np.array([20])
d["indexrange"]  = list(d["indexrange"].astype(int))


In [17]:
o = dict()

constants = constants_default.copy(); radios = radios_default.copy()
constants["XpointIndex"] = d["Xpoint"], # Index of Xpoint
constants["B"] = interpolate.interp1d(d["S"], d["Btot"], kind='cubic') # Continuous function for B

o["splot"],o["C"],o["Sprofiles"],o["Tprofiles"],o["Qprofiles"],o["log"] = returnImpurityFracLeng(constants,radios,d["S"],d["indexrange"])


Solving...0...12...25...38...51...64...76...89...102...115...128...140...153...166...179...Complete in 385.2 seconds


In [19]:
old = o

In [20]:
constants = constants_default.copy(); radios = radios_default.copy()
indexRange = np.linspace(0, d["Xpoint"], 15).astype(int)

radios["upstreamGrid"] = True
constants["XpointIndex"] = d["Xpoint"], # Index of Xpoint
constants["B"] = interpolate.interp1d(d["S"], d["Btot"], kind='cubic') # Continuous function for B
out = LRBv2(constants,radios,d["S"],d["indexrange"], 
                                    Ctol = 1e-3, Ttol = 1e-3, URF = 0.99, acceleration = 0, verbosity = 0,
                                    control_variable = "impurity_frac"
                )
out["SpolPlot"] = [d["Spol"][x] for x in indexRange]

new = out.copy()

Solving...0...12...25...38...51...64...76...89...102...115...128...140...153...166...179...Complete in 11.9 seconds


In [ ]:
fig, ax = plt.subplots(4,1,figsize = (15,10))
# ax.plot(log[i]["cz"], marker = "o", color = "black", markersize = 5, label = "cz")
ax[0].plot(log[i]["Tucalc"], marker = "o", color = "navy", markersize = 5, label = "Tucalc")
ax[1].plot(log[i]["Tu"], marker = "o", color = "darkorange", markersize = 5, label = "Tu")
ax[2].plot(log[i]["error1"], marker = "o", color = "purple", markersize = 5, label = "error1")
ax[3].plot(log[i]["error0"], marker = "o", color = "green", markersize = 5, label = "error0")

for a in ax:
    a.grid()
    a.legend()